In [ ]:
import pandas as pd

from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from sqlalchemy.orm import sessionmaker

from models import Team
from db_config import get_session
session = get_session()

In [ ]:
teams = session.query(Team).all()
session.close()

In [ ]:
team_ids = [team.id for team in teams]
season_type = "Regular Season"
season = "2023-24"

In [ ]:
def get_2023_2024_by_team_id(team_id):
    try:
        # Create the game finder instance with season and season type specified
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable=season, season_type_nullable=season_type)
        
        # Get the data and convert to DataFrame
        games_df = gamefinder.get_data_frames()[0]
        return games_df
        # Display the DataFrame
        print(games_df.head())
    except Exception as e:
        print(f"An error occurred: {e}")
        # Optionally, print out the raw API response for debugging
        print(gamefinder.get_json())



In [ ]:
test_games = get_2023_2024_by_team_id(team_ids[5])
print(test_games.columns)
print(test_games)

In [ ]:
games_by_teams = [get_2023_2024_by_team_id(team_id) for team_id in team_ids]
games = pd.concat(games_by_teams, axis=0)

In [ ]:
from datetime import datetime
from models import TeamStats, Game


for index, row in games.iterrows():
    game_date = datetime.strptime(row['GAME_DATE'], '%Y-%m-%d')

    # Check if the game already exists
    game = session.query(Game).filter_by(id=row['GAME_ID']).first()
    if not game:
        game = Game(id=row['GAME_ID'], date=game_date,
                    season = season, 
                    season_type = season_type,
                    home_team_id=row['TEAM_ID'] if 'vs.' in row['MATCHUP'] else None,
                    away_team_id=row['TEAM_ID'] if '@' in row['MATCHUP'] else None)
        session.add(game)

    # Check if TeamStats already exists for this game and team
    team_stats = session.query(TeamStats).filter_by(game_id=row['GAME_ID'], team_id=row['TEAM_ID']).first()
    if not team_stats:
        # Create new TeamStats if none exist
        team_stats = TeamStats(
            game=game,
            team_id=row['TEAM_ID']
        )
        session.add(team_stats)

    # Update existing or new TeamStats with current data
    team_stats.points=row['PTS']
    team_stats.field_goals_made=row['FGM']
    team_stats.field_goals_attempted=row['FGA']
    team_stats.field_goal_percentage=row['FG_PCT']
    team_stats.three_point_field_goals_made=row['FG3M']
    team_stats.three_point_field_goals_attempted=row['FG3A']
    team_stats.three_point_field_goal_percentage=row['FG3_PCT']
    team_stats.free_throws_made=row['FTM']
    team_stats.free_throws_attempted=row['FTA']
    team_stats.free_throw_percentage=row['FT_PCT']
    team_stats.offensive_rebounds=row['OREB']
    team_stats.defensive_rebounds=row['DREB']
    team_stats.total_rebounds=row['REB']
    team_stats.assists=row['AST']
    team_stats.steals=row['STL']
    team_stats.blocks=row['BLK']
    team_stats.turnovers=row['TOV']
    team_stats.personal_fouls=row['PF']
    team_stats.plus_minus=row['PLUS_MINUS']

session.commit()
